In [1]:
import dolfinx
import numpy as np
import ufl
import gmsh

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot, default_scalar_type
from dolfinx.io import gmshio
from dolfinx.io.gmshio import model_to_mesh
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc, LinearProblem

## Creating and Meshing the Domain

In [2]:
R1 = 0.5
R2 = 1

gmsh.initialize()

gmsh.model.occ.addPoint(0, R1, 0, 0.05, tag=1)
gmsh.model.occ.addPoint(R1, 0, 0, 0.05, tag=2)
gmsh.model.occ.addPoint(0, -R1, 0, 0.05, tag=3)
gmsh.model.occ.addPoint(0, R2, 0, 0.05, tag=4)
gmsh.model.occ.addPoint(R2, 0, 0, 0.05, tag=5)
gmsh.model.occ.addPoint(0, -R2, 0, 0.05, tag=6)

gmsh.model.occ.addLine(1, 4, tag=1)
gmsh.model.occ.addLine(6, 3, tag=3)

gmsh.model.occ.addCircleArc(4, 5, 6, tag=2, center=False)
gmsh.model.occ.addCircleArc(3, 2, 1, tag=4, center=False)

gmsh.model.occ.addCurveLoop([1, 2, 3, 4], tag=1)

gmsh.model.occ.addPlaneSurface([1], tag=1)

gmsh.model.occ.synchronize()

In [3]:
gdim = 2
gmsh.model.addPhysicalGroup(gdim, [1], tag=1)
gmsh.model.mesh.generate(gdim)
# gmsh.write("mesh.msh")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Circle)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Circle)
Info    : Done meshing 1D (Wall 0.000704742s, CPU 0.001358s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0315427s, CPU 0.032021s)
Info    : 648 nodes 1296 elements


In [4]:
model_rank = 0
domain, cell_tags, facet_tags = model_to_mesh(gmsh.model, MPI.COMM_WORLD, model_rank)
V_sc = fem.functionspace(domain, ("Lagrange", 1))
V_vec = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

## Model setup

In [5]:
# Define temporal parameters
t = 0  # Start time
T = 2  # Final time
num_steps = 1000
dt = T / num_steps  # time step size

In [6]:
# Create initial condition
def initial_condition(x, a=0.1):
    r = np.sqrt(x[0]**2 + x[1]**2) - R1
    return 1 + 0.1*np.exp(-a * (r**2))

n_n = fem.Function(V_sc)
n_n.name = "Cells"
n_n.interpolate(initial_condition)

p_n = fem.Function(V_sc)
p_n.name = "Collagen"
p_n.interpolate(initial_condition)

u_n = fem.Function(V_vec)
u_n.name = "Deformation"

vel_n = fem.Function(V_vec)
vel_n.name = "Velocity"

In [7]:
# Create boundary condition
def inner_boundary(x):
    r = np.sqrt(x[0]**2 + x[1]**2)
    inner_arc = np.isclose(r,R1)
    inner_lines = np.isclose(x[0],0)
    return np.logical_or(inner_arc, inner_lines)

def inner_lines(x):
    return np.isclose(x[0],0)

def outer_arc(x):
    r = np.sqrt(x[0]**2 + x[1]**2)
    return np.isclose(r, R2)

fdim = domain.topology.dim - 1
inner_boundary_facets = mesh.locate_entities_boundary(domain, fdim, inner_boundary)
inner_lines = mesh.locate_entities_boundary(domain, fdim, inner_lines)
outer_arc = mesh.locate_entities_boundary(domain, fdim, outer_arc)
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))

u_D = np.array([0, 0, 0], dtype=default_scalar_type)
bc_u = fem.dirichletbc(u_D, fem.locate_dofs_topological(V_vec, fdim, inner_lines), V_vec)

bc_n = fem.dirichletbc(PETSc.ScalarType(1), fem.locate_dofs_topological(V_sc, fdim, outer_arc), V_sc)

bc_p = fem.dirichletbc(PETSc.ScalarType(1), fem.locate_dofs_topological(V_sc, fdim, outer_arc), V_sc)

## Time Dependent Output

In [8]:
xdmf = io.XDMFFile(domain.comm, "coupled.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
nh = fem.Function(V_sc)
nh.name = "Cells"
nh.interpolate(initial_condition)
xdmf.write_function(nh, t)

ph = fem.Function(V_sc)
ph.name = "Collagen"
ph.interpolate(initial_condition)
xdmf.write_function(ph, t)

# Function with all zero entries by default
uh = fem.Function(V_vec)
uh.name = "Deformation"
# xdmf.write_function(uh, t)

In [9]:
n, v_n = ufl.TrialFunction(V_sc), ufl.TestFunction(V_sc)
p, v_p = ufl.TrialFunction(V_sc), ufl.TestFunction(V_sc)
u, v_u = ufl.TrialFunction(V_vec), ufl.TestFunction(V_vec)

In [10]:
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

In [11]:
ds = ufl.Measure("ds", domain=domain)

In [12]:
nabla = 1.25
mu = 1
mu_1 = 1
mu_2 = 1
x = ufl.SpatialCoordinate(domain)

def epsilon(u):
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)

def sigma(u):
    return nabla * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)

def sigma_v(vel):
    return mu_1 * epsilon(vel) + mu_2 * ufl.nabla_div(vel) * ufl.Identity(len(vel))

In [13]:
vel_init = vel_n

for i in range(num_steps):
    t += dt

    tau = 0.3
    f_n = fem.Constant(domain, PETSc.ScalarType(1)) 
    sig = sigma(uh) + sigma_v(vel_n) + tau*nh*ph*ufl.Identity(len(x))
    f_sig = ufl.tr(sig)
    
    a_n = ((1 + dt) * ufl.inner(n, v_n) * ufl.dx +
    dt * ufl.inner(ufl.grad(n), ufl.grad(v_n)) * ufl.dx +
    dt * ufl.inner(ufl.div(n * vel_n), v_n) * ufl.dx)
    L_n = ufl.inner((n_n + dt * f_n), v_n) * ufl.dx + ufl.inner(dt * f_sig, v_n) * ufl.dx

    bilinear_form = fem.form(a_n)
    linear_form = fem.form(L_n)
    A = assemble_matrix(bilinear_form, bcs=[bc_n])
    A.assemble()
    b = create_vector(linear_form)

    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(PETSc.KSP.Type.PREONLY)
    solver.getPC().setType(PETSc.PC.Type.LU)

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc_n]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc_n])

    # Solve linear problem
    solver.solve(b, nh.x.petsc_vec)
    nh.x.scatter_forward()

    # Update solution at previous time step (n_n)
    n_n.x.array[:] = nh.x.array

    # Equation for Collagen
    f_p = nh 
    a_p = (1 + dt) * ufl.inner(p, v_p) * ufl.dx + dt * ufl.inner(ufl.div(p*vel_n), v_p) * ufl.dx
    L_p = ufl.inner((p_n + dt * f_p), v_p) * ufl.dx

    bilinear_form = fem.form(a_p)
    linear_form = fem.form(L_n)
    A = assemble_matrix(bilinear_form, bcs=[bc_p])
    A.assemble()
    b = create_vector(linear_form)

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc_p]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc_p])

    # Solve linear problem
    solver.solve(b, ph.x.petsc_vec)
    ph.x.scatter_forward()

    # Update solution at previous time step (n_n)
    p_n.x.array[:] = ph.x.array

    # Solve deformation problem
    traction = tau*nh*ph*ufl.Identity(len(x))
    f_u = ufl.nabla_div(traction)
    a_u = ufl.inner(sigma(u), epsilon(v_u)) * ufl.dx
    L_u = ufl.inner(f_u, v_u) * ufl.dx + ufl.inner(T, v_u) * ds - ufl.inner(sigma_v(vel_n), epsilon(v_u)) * ufl.dx
    problem = LinearProblem(a_u, L_u, bcs=[bc_u], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    uh = problem.solve()

    # Compute velocity
    if i % 10 == 0 and i != 0:
        vel_n.x.array[:] = (uh.x.array[:] - u_n.x.array[:])/dt
    # if i != 0:    
    #     vel_n.x.array[:] = (uh.x.array[:] - u_n.x.array[:])/dt
    
    print(i, (uh.sub(0).x.array[100]- u_n.sub(0).x.array[100])/dt)
    
    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array
    # print(uh.sub(0).x.array[100])

    # Write solution to file
    nh.name = "Cells"
    ph.name = "Collagen"
    uh.name = "Deformation"
    xdmf.write_function(nh, t)
    xdmf.write_function(ph, t)
    xdmf.write_function(uh, t)

    # Write velocity
    velocity_expr = fem.Expression(vel_n, V_vec.element.interpolation_points())
    velocity = fem.Function(V_vec)
    velocity.interpolate(velocity_expr)
    velocity.name = "velocity"
    xdmf.write_function(velocity, t)

    # Compute and write stressn % k == 0
    s = sigma(uh) - 1. / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
    von_Mises = ufl.sqrt(3. / 2 * ufl.inner(s, s))
    V_von_mises = fem.functionspace(domain, ("DG", 0))
    stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
    stresses = fem.Function(V_von_mises)
    stresses.interpolate(stress_expr)
    stresses.name = "stress"
    xdmf.write_function(stresses, t)
xdmf.close()

0 (3.175245834481907+0j)
1 (-0.10841684415326688+0j)
2 (-0.07234387542095133+0j)
3 (-0.0562645036318722+0j)
4 (-0.04576535733756375+0j)
5 (-0.0388958045206755+0j)
6 (-0.03445349215975982+0j)
7 (-0.031635200297436966+0j)
8 (-0.029883302685483962+0j)
9 (-0.028809111612490383+0j)
10 (-0.028145766537155618+0j)
11 (9.184070179906557+0j)
12 (-0.049992461013670386+0j)
13 (-0.03444792330515624+0j)
14 (-0.03384093409243907+0j)
15 (-0.033280772074558915+0j)
16 (-0.03272573541235162+0j)
17 (-0.032157398412491905+0j)
18 (-0.03156828751465883+0j)
19 (-0.030956886820154156+0j)
20 (-0.030324891247355407+0j)
21 (0.3427993021181165+0j)
22 (-0.041889546731697924+0j)
23 (-0.031831807927998335+0j)
24 (-0.030905391594648543+0j)
25 (-0.03001567425161243+0j)
26 (-0.0291506303404538+0j)
27 (-0.028307817178701428+0j)
28 (-0.0274868408104232+0j)
29 (-0.026687715049459812+0j)
30 (-0.025910431764078132+0j)
31 (-1.5564637595426016+0j)
32 (-0.020188513899569577+0j)
33 (-0.022322783594073276+0j)
34 (-0.0216948596708